In [1]:
import subprocess
import pandas as pd
import numpy as np
import random
from helper_fns import *
from sklearn.metrics import r2_score
random.seed(42)

In [2]:
EUR_ge_regressed, YRI_ge_regressed, EUR_protein_genes, YRI_protein_genes = load_data()

Shapes of the dataframes: (373, 13942) (89, 13942) (13942, 5) (13942, 5)


In [3]:
# Function to extract SNPs for a gene and split into train/test
def process_gene(gene, chr, start, end, ancestry="EUR", plink_path='plink'):
    if start < 0:
        return
    base_name = f'{gene}_chr{chr}_{start}_{end}'
    if ancestry == "EUR":
        bfile_path = "/new-stg/home/banghua/TWAS_ASSOC/project_data/geno/EUR/GEUVADIS_EUR_chr" + str(chr)
    else:
        bfile_path = "/new-stg/home/banghua/TWAS_ASSOC/project_data/geno/YRI/GEUVADIS_YRI_chr" + str(chr)

    if ancestry == "EUR":
        gene_expression = EUR_ge_regressed
    else:
        gene_expression = YRI_ge_regressed

    gene_exp_one_gene = gene_expression[[gene]]

    # Step 2: Extract gene-specific SNPs
    subprocess.run([plink_path, '--bfile', bfile_path, '--chr', str(chr), 
                    '--from-bp', str(start), '--to-bp', str(end), 
                    '--make-bed', '--out', base_name])
    
    # Step 3: Add gene expression data
    fam = pd.read_csv(f"/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/{base_name}.fam",
                      sep='\s+', header=None)
    all_individuals = fam[[1]].values.flatten().tolist()
    fam.index = all_individuals
    print(all_individuals)
    gene_exp_one_gene = gene_exp_one_gene.loc[all_individuals]
    print(gene_exp_one_gene)
    # Add gene expression to fam
    fam[[5]] = gene_exp_one_gene
    print(fam)
    fam.to_csv(f"/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/{base_name}.fam",
                sep=' ', index=False, header=False)

    # Count the number of individuals
    with open(f'{base_name}.fam') as f:
        num_individuals = sum(1 for line in f)
    
    # Calculate split
    train_num = int(num_individuals * 0.8)
    
    # Shuffle individuals
    individuals = pd.read_csv(f'{base_name}.fam', sep='\s+', header=None)
    shuffled = individuals.sample(frac=1).reset_index(drop=True)
    
    # Split into train/test
    train = shuffled.head(train_num)
    test = shuffled.tail(num_individuals - train_num)
    
    train[[0, 1]].to_csv(f'{base_name}_train.txt', sep=' ', index=False, header=False)
    test[[0, 1]].to_csv(f'{base_name}_test.txt', sep=' ', index=False, header=False)
    
    # Generate train/test datasets
    subprocess.run([plink_path, '--bfile', base_name, '--keep', f'{base_name}_train.txt', 
                    '--make-bed', '--out', f'{base_name}_train'])
    subprocess.run([plink_path, '--bfile', base_name, '--keep', f'{base_name}_test.txt', 
                    '--make-bed', '--out', f'{base_name}_test'])
    
    return fam

In [4]:
os.chdir("/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific")

In [5]:
gene_info = pd.read_csv("/new-stg/home/banghua/TWAS_ASSOC/EUR_protein_genes.csv")

In [6]:
gene_info.head()

,start,end,gene_id,name,chr
0,360260,1360261,ENSG00000187634,SAMD11,1
1,379584,1379585,ENSG00000188976,NOC2L,1
2,395967,1395968,ENSG00000187961,KLHL17,1
3,401877,1401878,ENSG00000187583,PLEKHN1,1
4,410579,1410580,ENSG00000187642,PERM1,1


In [7]:
gene_info[gene_info["gene_id"] == "ENSG00000174885"]

,start,end,gene_id,name,chr
2053,-221635,778366,ENSG00000174885,NLRP6,11


In [23]:
for index, row in gene_info.iterrows():
    gene = row['gene_id']
    chr = row['chr']
    start = row['start']
    end = row['end']
    print(f'Processing {gene} on chr{chr} from {start} to {end}')
    fam = process_gene(gene, chr, start, end)
    break

Processing ENSG00000187634 on chr1 from 360260 to 1360261
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ENSG00000187634_chr1_360260_1360261.log.
Options in effect:
  --bfile /new-stg/home/banghua/TWAS_ASSOC/project_data/geno/EUR/GEUVADIS_EUR_chr1
  --chr 1
  --from-bp 360260
  --make-bed
  --out ENSG00000187634_chr1_360260_1360261
  --to-bp 1360261

63755 MB RAM detected; reserving 31877 MB for main workspace.
137 out of 85764 variants loaded from .bim file.
373 people (0 males, 0 females, 373 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ENSG00000187634_chr1_360260_1360261.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 373 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676

phenotypes to be ignored, use the --allow-no-sex flag.
phenotypes to be ignored, use the --allow-no-sex flag.


In [10]:
fam = pd.read_csv(f"/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/{base_name}.fam",
                   sep='\s+', header=None)

,0,1,2,3,4,5
0,HG00096,HG00096,0,0,0,NaN
1,HG00097,HG00097,0,0,0,NaN
2,HG00099,HG00099,0,0,0,NaN
3,HG00100,HG00100,0,0,0,NaN
4,HG00101,HG00101,0,0,0,NaN
...,...,...,...,...,...,...
368,NA20815,NA20815,0,0,0,NaN
369,NA20816,NA20816,0,0,0,NaN
370,NA20819,NA20819,0,0,0,NaN
371,NA20826,NA20826,0,0,0,NaN


In [ ]:
process_gene('ENSG00000187634', 1, 360260, 1360261)

In [12]:
fam = pd.read_csv("/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/ENSG00000187634_chr1_360260_1360261.fam",
                  sep='\s+', header=None)

In [13]:
fam

,0,1,2,3,4
0,HG00096,HG00096,0,0,0
1,HG00097,HG00097,0,0,0
2,HG00099,HG00099,0,0,0
3,HG00100,HG00100,0,0,0
4,HG00101,HG00101,0,0,0
...,...,...,...,...,...
368,NA20815,NA20815,0,0,0
369,NA20816,NA20816,0,0,0
370,NA20819,NA20819,0,0,0
371,NA20826,NA20826,0,0,0


In [17]:
all_individuals = fam[[1]].values.flatten().tolist()

In [8]:
import os

In [9]:
os.listdir("/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/EUR")

['ENSG00000073067',
 'ENSG00000196659',
 'ENSG00000105647',
 'ENSG00000109066',
 'ENSG00000081177',
 'ENSG00000079337',
 'ENSG00000101596',
 'ENSG00000218739',
 'ENSG00000184828',
 'ENSG00000164749',
 'ENSG00000153531',
 'ENSG00000196754',
 'ENSG00000184508',
 'ENSG00000145425',
 'ENSG00000160401',
 'ENSG00000119403',
 'ENSG00000138615',
 'ENSG00000166170',
 'ENSG00000184925',
 'ENSG00000048544',
 'ENSG00000149273',
 'ENSG00000115255',
 'ENSG00000256980',
 'ENSG00000137168',
 'ENSG00000129515',
 'ENSG00000152223',
 'ENSG00000092439',
 'ENSG00000100284',
 'ENSG00000156011',
 'ENSG00000183785',
 'ENSG00000082126',
 'ENSG00000108774',
 'ENSG00000129562',
 'ENSG00000148841',
 'ENSG00000078808',
 'ENSG00000175544',
 'ENSG00000114867',
 'ENSG00000187559',
 'ENSG00000116273',
 'ENSG00000171798',
 'ENSG00000165156',
 'ENSG00000234511',
 'ENSG00000152359',
 'ENSG00000140105',
 'ENSG00000146859',
 'ENSG00000131480',
 'ENSG00000175449',
 'ENSG00000102890',
 'ENSG00000088298',
 'ENSG00000197046',


In [51]:
def weights_bslmm(input, bv_type, snp, out=None, gemma_path="gemma", sys_print=False):
    """
    Run BSLMM analysis using GEMMA and extract effect sizes for specified SNPs.

    Parameters:
    - input: Base name for input files.
    - bv_type: Specifies the type of BSLMM analysis.
    - snp: List or array of SNP identifiers for which weights are to be calculated.
    - out: Optional. Specifies the base name for output files. Defaults to None.
    - gemma_path: Path to the GEMMA executable. Defaults to 'gemma'.
    - sys_print: If True, prints the GEMMA command output.

    Returns:
    - A numpy array of effect weights for the input SNPs.
    """
    if out is None:
        out = f"{input}.BSLMM"

    # # Constructing the GEMMA command
    arg = f"{gemma_path} -miss 1 -maf 0 -r2 1 -rpace 1000 -wpace 1000 -bfile {input} -bslmm {bv_type} -o {out}"

    # Execute the GEMMA command
    result = subprocess.run(arg, shell=True, capture_output=not sys_print)
    if not sys_print:
        print(result.stdout.decode())  # Optional: print GEMMA output for debugging.

    # Read the output parameter file
    try:
        eff = pd.read_table(f"./output/{out}.param.txt", header=0, sep='\t')
    except FileNotFoundError:
        raise FileNotFoundError("GEMMA output file not found. Check GEMMA execution and output path.")

    # Initialize effect weights with NaN for all SNPs
    eff_wgt = pd.Series(np.nan, index=snp)

    # Match SNPs and assign weights
    for i, snp_id in enumerate(snp):
        if snp_id in eff['rs'].values:
            row = eff.loc[eff['rs'] == snp_id].iloc[0]
            eff_wgt.at[snp_id] = row['alpha'] + row['beta'] * row['gamma']

    return eff_wgt

In [10]:
os.chdir("/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/EUR/ENSG00000000419")

In [54]:
def train_pred_bslmm(bfile, bv_type, out=None, gemma_path="gemma", sys_print=False):
    os.chdir("/new-stg/home/banghua/TWAS_ASSOC/project_data/geno_gene_specific/EUR/ENSG00000000419")
    bfile_train = f"{bfile}_train"
    bfile_test = f"{bfile}_test"
    with PyPlink(bfile_train) as bed:
        # Get SNP names
        bim = bed.get_bim()
        snps = bim.index.tolist()
        fam = bed.get_fam()
        # Get effect weights
        weights = weights_bslmm(bfile_train, bv_type, snps, out, gemma_path, sys_print)
        weights_unfiltered = weights
        train_y = fam[["status"]].values.flatten()
        X = np.zeros((len(train_y), len(snps)))

        for snp_id, genotypes in bed.iter_geno_marker(snps):
            X[:, snps.index(snp_id)] = genotypes

        # Find index of nan in weights
        nan_idx = np.argwhere(np.isnan(weights)).flatten()
        # Remove nan from weights and X
        weights = np.delete(weights, nan_idx)
        X = np.delete(X, nan_idx, axis=1)
        train_pred_y = np.dot(X, weights)
        train_r2 = r2_score(train_y, train_pred_y)

    with PyPlink(bfile_test) as bed:
        # Get SNP names
        bim = bed.get_bim()
        snps = bim.index.tolist()
        fam = bed.get_fam()

        test_y = fam[["status"]].values.flatten()
        X = np.zeros((len(test_y), len(snps)))

        for snp_id, genotypes in bed.iter_geno_marker(snps):
            X[:, snps.index(snp_id)] = genotypes
        X = np.delete(X, nan_idx, axis=1)
        test_pred_y = np.dot(X, weights)
        test_r2 = r2_score(test_y, test_pred_y)

    return {
        "eff_weights": weights,
        "eff_weights_unfiltered": weights_unfiltered,
        "train_r2": train_r2,
        "test_r2": test_r2
    }

In [55]:
result = train_pred_bslmm("ENSG00000000419_chr20_49051404_50051405", 1, sys_print=True)

GEMMA 0.98.3 (2020-11-28) by Xiang Zhou and team (C) 2012-2020
Reading Files ... 
## number of total individuals = 298
## number of analyzed individuals = 298
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =      424
## number of analyzed SNPs         =      422
Start Eigen-Decomposition...
pve estimate =3.29542e-06
se(pve) =0.035648
Calculating UtX...
initial value of h = 3.29542e-06
initial value of rho = 1
initial value of pi = 0.0236967
initial value of |gamma| = 10
================================================== 100%    0.19


**** INFO: Done.
